In [34]:
from ddsm import noise_factory
import torch

In [35]:
num_cat = 9
num_samples = 500000
num_time_steps = 100
max_time = 4.0
order = 1000
steps_per_tick = 200
logspace = True
speed_balance = True #if True s=2/(a+b) is used; otherwise s=1
mode ='independent'
device="cpu"
boundary_mode='clamp' #choose from 'clamp', 'reflect_boundaries', 'reflect'

In [36]:
alpha = torch.ones(1)
beta =  (num_cat-1)*torch.ones(1)

In [37]:
print(alpha.shape)
print(beta.shape)

torch.Size([1])
torch.Size([1])


In [ ]:
v_one, v_zero = noise_factory(num_samples, 
                                num_time_steps,
                                alpha,
                                beta,
                                total_time=max_time,
                                order=order,
                                time_steps=steps_per_tick,
                                logspace=logspace,
                                speed_balanced=speed_balance,
                                mode=mode,
                                device=device,
                                noise_only=True)

i=0
i=1
i=2
i=3
i=4
i=5
i=6
i=7
i=8
i=9
i=10
i=11
i=12
i=13
i=14
i=15
i=16
i=17
i=18
i=19
i=20
i=21
i=22
i=23
i=24
i=25
i=26
i=27
i=28
i=29
i=30
i=31
i=32
i=33
i=34
i=35
i=36
i=37
i=38
i=39
i=40
i=41
i=42
i=43
i=44
i=45
i=46
i=47
i=48
i=49
i=50
i=51
i=52
i=53
i=54
i=55
i=56
i=57
i=58
i=59
i=60
i=61
i=62
i=63
i=64
i=65
i=66
i=67
i=68
i=69
i=70
i=71
i=72
i=73
i=74
i=75
i=76
i=77
i=78
i=79
i=80
i=81
i=82
i=83
i=84
i=85
i=86
i=87
i=88
i=89
i=90


In [6]:
print(v_one.shape)
print(v_zero.shape)

torch.Size([1000, 400, 1])
torch.Size([1000, 400, 1])


In [7]:
from scipy.stats import wasserstein_distance as wsd
from scipy.stats import beta as beta_distribution

In [8]:
vT = torch.cat([v_one[:,-1,0],v_zero[:,-1,0]])
vT_true = beta_distribution.rvs(alpha, beta, size=vT.shape[0])
d = wsd(vT,vT_true)

In [9]:
print(d)

0.010728219828756803


## Ground truth

In [74]:
iteration = 10
distances = torch.zeros(iteration)
for i in range(iteration):
    test1 = beta_distribution.rvs(alpha, beta, size=1000000)
    test2 = beta_distribution.rvs(alpha, beta, size=1000000)
    d = wsd(test1,test2)
    print('distance',d)
    distances[i] = d

test1 (1000000,)
test2 (1000000,)
distance 0.00015255260316868874
test1 (1000000,)
test2 (1000000,)
distance 0.00021186835176490015
test1 (1000000,)
test2 (1000000,)
distance 0.00011741920541712615
test1 (1000000,)
test2 (1000000,)
distance 0.00019672195663265597
test1 (1000000,)
test2 (1000000,)
distance 0.00014337657853888966
test1 (1000000,)
test2 (1000000,)
distance 0.00022759701250514168
test1 (1000000,)
test2 (1000000,)
distance 0.00029576271299861255
test1 (1000000,)
test2 (1000000,)
distance 0.00010474523625257931
test1 (1000000,)
test2 (1000000,)
distance 0.0001795923039257112
test1 (1000000,)
test2 (1000000,)
distance 0.00011724995625918759


In [75]:
print('mean',torch.mean(distances))
print('std',torch.std(distances))

mean tensor(0.0002)
std tensor(5.9900e-05)
